**1. How deep learning works at all**

To figure out how deep learning works, it's not sufficient to focus on the loss function or the model class only. During training step, many algorithms are used to find minima namely: Gradient Descente (GD), Mini-Batch Gradient Descente (MBGD) or Stochastic Gradient Descente (SGD). But SGD seems to play an important role.

For the specific training data, there are several minima, some of them generalise well (ie result in low test error) others can be arbitrarly badly overfit.

In this kind of situation, one is interested in whether the optimisation algorithm converges quickly to a local minimum (such as a general principle) but here the interest is in which of the available minima it prefers to reach first. 

Optimisation algorithms have certain preference in their convergence to a minimum among the possible available minima and this preference is often described as an "implicit regularization".

**2. State of the art**

When we train the deep learning model, the learning rate plays in some case an important role. Managing the learning rate can significantly achieve the performance both in test and train accuracies. Large learning rate can give the high test accuracy and this effect can minimize the training loss. It's often difficult to generalize this phenomenom, since the learning rate which maximizes test accuracy is often larger than the learning rate which minimizes training loss.

To interpret this phenomenom, we use the SGD by modifying the loss function. 

This modification consists to combine the original loss function with an implicit regularizer which penalizes the norms of the Mini-Batch Gradients. All this modification is done under some assumptions, when the batch size is small the scale of the implicit regularization term is proportional to the ratio of the learning rate to the batch size.

The modified loss is:

$$C_{modSGD}(w) = C_{Org}(w) + \dfrac{\epsilon}{4m}\displaystyle\sum_{j = 1}^{m}\|{\nabla C_j(w)}\|^2$$

Where $m = \dfrac{N}{B}$ defines the number of batches per epoch, B: batch size and N: The training set size.

**3. Core idea**

Modified the loss function in order to see how the small and finite learning rate can aid generalization.

This novel approach is called **implicit regularization**, this technique is established for analysing optimization algorithms (especially SGD) with finite and small step or learning rate. SGD with random shuffling, iterate also stays close to the path of gradient flow if the learning rate is small and finite, but on a modified loss.

**4. How to approach the paper in term of the implementation ?**



In [23]:
from __future__ import print_function
import mxnet as mx
import mxnet.ndarray as nd
from mxnet import nd, autograd, gluon
import numpy as np
ctx = mx.cpu()
mx.random.seed(1)


# for plotting purposes
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

In [24]:
batch_size = 64
num_inputs = 784
num_outputs = 10
def transform(data, label):
    return nd.transpose(data.astype(np.float32), (2,0,1))/255, label.astype(np.float32)
train_data = gluon.data.DataLoader(gluon.data.vision.MNIST(train=True, transform=transform),
                                      batch_size, shuffle=True)
test_data = gluon.data.DataLoader(gluon.data.vision.MNIST(train=False, transform=transform),
                                     batch_size, shuffle=False)

In [26]:
# Parameters

#######################
#  Set the scale for weight initialization and choose
#  the number of hidden units in the fully-connected layer
#######################

weight_scale = .01
num_fc = 128
num_filter_conv_layer1 = 20
num_filter_conv_layer2 = 50

W1 = nd.random_normal(shape=(num_filter_conv_layer1, 1, 3,3), scale=weight_scale, ctx=ctx)
b1 = nd.random_normal(shape=num_filter_conv_layer1, scale=weight_scale, ctx=ctx)

W2 = nd.random_normal(shape=(num_filter_conv_layer2, num_filter_conv_layer1, 5, 5),
                                                    scale=weight_scale, ctx=ctx)
b2 = nd.random_normal(shape=num_filter_conv_layer2, scale=weight_scale, ctx=ctx)

W3 = nd.random_normal(shape=(800, num_fc), scale=weight_scale, ctx=ctx)
b3 = nd.random_normal(shape=num_fc, scale=weight_scale, ctx=ctx)

W4 = nd.random_normal(shape=(num_fc, num_outputs), scale=weight_scale, ctx=ctx)
b4 = nd.random_normal(shape=num_outputs, scale=weight_scale, ctx=ctx)

params = [W1, b1, W2, b2, W3, b3, W4, b4]

MXNetError: Traceback (most recent call last):
  File "C:\Jenkins\workspace\mxnet-tag\mxnet\src\resource.cc", line 167
MXNetError: GPU is not enabled

In [ ]:
for param in params:
    param.attach_grad()

In [ ]:

for data, _ in train_data:
    data = data.as_in_context(ctx)
    break
conv = nd.Convolution(data=data, weight=W1, bias=b1, kernel=(3,3), num_filter=num_filter_conv_layer1)
print(conv.shape)

(64, 20, 26, 26)


In [ ]:
# Average pooling

pool = nd.Pooling(data=conv, pool_type="max", kernel=(2,2), stride=(2,2))
print(pool.shape)

(64, 20, 13, 13)


In [ ]:
# Activation function

def relu(X):
    return nd.maximum(X,nd.zeros_like(X))

In [ ]:
# Softmax output

def softmax(y_linear):
    exp = nd.exp(y_linear-nd.max(y_linear))
    partition = nd.sum(exp, axis=0, exclude=True).reshape((-1,1))
    return exp / partition

In [ ]:
# Softmax cross-entropy loss

def softmax_cross_entropy(yhat_linear, y):
    return - nd.nansum(y * nd.log_softmax(yhat_linear), axis=0, exclude=True)

In [ ]:
# # Model

# import torch.nn as nn
# import torch.nn.functional as F


# class Net(nn.Module):
#     def __init__(self):
#         super(Net, self).__init__()
#         self.conv1 = nn.Conv2d(3, 6, 5)
#         self.pool = nn.MaxPool2d(2, 2)
#         self.conv2 = nn.Conv2d(6, 16, 5)
#         self.fc1 = nn.Linear(16 * 5 * 5, 120)
#         self.fc2 = nn.Linear(120, 84)
#         self.fc3 = nn.Linear(84, 10)

#     def forward(self, x):
#         x = self.pool(F.relu(self.conv1(x)))
#         x = self.pool(F.relu(self.conv2(x)))
#         x = x.view(-1, 16 * 5 * 5)
#         x = F.relu(self.fc1(x))
#         x = F.relu(self.fc2(x))
#         x = self.fc3(x)
#         return x


# net = Net()

In [ ]:
# model

def net(X, debug=False):
    ########################
    #  Define the computation of the first convolutional layer
    ########################
    h1_conv = nd.Convolution(data=X, weight=W1, bias=b1, kernel=(3,3),
                                  num_filter=num_filter_conv_layer1)
    h1_activation = relu(h1_conv)
    h1 = nd.Pooling(data=h1_activation, pool_type="avg", kernel=(2,2), stride=(2,2))
    if debug:
        print("h1 shape: %s" % (np.array(h1.shape)))

    ########################
    #  Define the computation of the second convolutional layer
    ########################
    h2_conv = nd.Convolution(data=h1, weight=W2, bias=b2, kernel=(5,5),
                                  num_filter=num_filter_conv_layer2)
    h2_activation = relu(h2_conv)
    h2 = nd.Pooling(data=h2_activation, pool_type="avg", kernel=(2,2), stride=(2,2))
    if debug:
        print("h2 shape: %s" % (np.array(h2.shape)))

    ########################
    #  Flattening h2 so that we can feed it into a fully-connected layer
    ########################
    h2 = nd.flatten(h2)
    if debug:
        print("Flat h2 shape: %s" % (np.array(h2.shape)))

    ########################
    #  Define the computation of the third (fully-connected) layer
    ########################
    h3_linear = nd.dot(h2, W3) + b3
    h3 = relu(h3_linear)
    if debug:
        print("h3 shape: %s" % (np.array(h3.shape)))

    ########################
    #  Define the computation of the output layer
    ########################
    yhat_linear = nd.dot(h3, W4) + b4
    #yhat_linear = softmax(yhat_linear)
    if debug:
        print("yhat_linear shape: %s" % (np.array(yhat_linear.shape)))

    return yhat_linear

In [ ]:
# Test Run 

output = net(data, debug=True)

h1 shape: [64 20 13 13]
h2 shape: [64 50  4  4]
Flat h2 shape: [ 64 800]
h3 shape: [ 64 128]
yhat_linear shape: [64 10]


In [ ]:
# Optimizer

def SGD(params, lr):
    for param in params:
        param[:] = param - lr * param.grad

In [ ]:
# The Metric Evaluation

def evaluate_accuracy(data_iterator, net):
    numerator = 0.
    denominator = 0.
    loss_avg = 0.
    for i, (data, label) in enumerate(data_iterator):
        data = data.as_in_context(ctx)
        label = label.as_in_context(ctx)
        label_one_hot = nd.one_hot(label, 10)
        output = net(data)
        predictions = nd.argmax(output, axis=1)
        numerator += nd.sum(predictions == label)
        denominator += data.shape[0]
    return (numerator / denominator).asscalar()

In [ ]:
# def evaluate_accuracy(data_iterator, net):
#     numerator = 0.
#     denominator = 0.
#     loss_avg = 0.
#     for i, (data, label) in enumerate(data_iterator):
#         data = data.as_in_context(ctx).reshape((-1,784))
#         label = label.as_in_context(ctx)
#         label_one_hot = nd.one_hot(label, 10)
#         output = net(data)
#         loss = cross_entropy(output, label_one_hot)
#         predictions = nd.argmax(output, axis=1)
#         numerator += nd.sum(predictions == label)
#         denominator += data.shape[0]
#         loss_avg = loss_avg*i/(i+1) + nd.mean(loss).asscalar()/(i+1)
#     return (numerator / denominator).asscalar(), loss_avg


In [ ]:
# Training Loop

epochs = 5
learning_rate = .01
smoothing_constant = .01

for e in range(epochs):
    for i, (data, label) in enumerate(train_data):
        data = data.as_in_context(ctx)
        label = label.as_in_context(ctx)
        label_one_hot = nd.one_hot(label, num_outputs)
        with autograd.record():
            output = net(data)
            loss = softmax_cross_entropy(output, label_one_hot)
        loss.backward()
        SGD(params, learning_rate)

        ##########################
        #  Keep a moving average of the losses
        ##########################
        curr_loss = nd.mean(loss).asscalar()
        moving_loss = (curr_loss if ((i == 0) and (e == 0))
                       else (1 - smoothing_constant) * moving_loss + (smoothing_constant) * curr_loss)


    test_accuracy = evaluate_accuracy(test_data, net)
    train_accuracy = evaluate_accuracy(train_data, net)
    print("Epoch %s. Loss: %s, Train_acc %s, Test_acc %s" % (e, moving_loss, train_accuracy, test_accuracy))

KeyboardInterrupt: 

*Implicit regularization*

In [ ]:
# implicit regularization

batch = len(train_data) / batch_size

def l2_penalty(params):
    penalty = nd.zeros(shape=1)
    for param in params:
        penalty = penalty + (1 / 4 * batch) * nd.sum(nd.norm(param.grad, ord = 2) ** 2)
    return penalty

In [ ]:
# for param in params:
#     param[:] = nd.random_normal(shape=param.shape)

In [ ]:
# Training Loop

epochs = 5
learning_rate = 2e-2
smoothing_constant = .01
#l2_strength = learning_rate / (4 * batch_size)
l2_strength = 2e-2

for e in range(epochs):
    for i, (data, label) in enumerate(train_data):
        data = data.as_in_context(ctx)
        label = label.as_in_context(ctx)
        label_one_hot = nd.one_hot(label, num_outputs)
        with autograd.record():
            output = net(data)
            loss = softmax_cross_entropy(output, label_one_hot) + l2_strength * l2_penalty(params)
        loss.backward()
        SGD(params, learning_rate)

        ##########################
        #  Keep a moving average of the losses
        ##########################
        curr_loss = nd.mean(loss).asscalar()
        moving_loss = (curr_loss if ((i == 0) and (e == 0))
                       else (1 - smoothing_constant) * moving_loss + (smoothing_constant) * curr_loss)


    test_accuracy = evaluate_accuracy(test_data, net)
    train_accuracy = evaluate_accuracy(train_data, net)
    print("Epoch %s. Loss: %s, Train_acc %s, Test_acc %s" % (e, moving_loss, train_accuracy, test_accuracy))

Epoch 0. Loss: 33.35400507958277, Train_acc 0.11236667, Test_acc 0.1135
Epoch 1. Loss: 34.8734357119291, Train_acc 0.09871667, Test_acc 0.098
Epoch 2. Loss: 34.24208737771534, Train_acc 0.09751666, Test_acc 0.0974
Epoch 3. Loss: 33.67863074317033, Train_acc 0.09751666, Test_acc 0.0974
Epoch 4. Loss: 34.30400169363496, Train_acc 0.11236667, Test_acc 0.1135
